In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, BooleanType, TimestampType, DateType


def clean_common(df):
    # Trim string columns
    string_cols = [f.name for f in df.schema.fields if f.dataType == StringType()]
    for col_name in string_cols:
        df = df.withColumn(col_name, trim(col(col_name)))
    
    # Replace "null", "n/a", "N/A", "" with actual nulls
    for col_name in df.columns:
        df = df.withColumn(
            col_name, 
            when(col(col_name).cast("string").isin("null", "n/a", "N/A", ""), None)
            .otherwise(col(col_name))
        )

    return df


# Define paths
storage_account_name = "adcampaigndata"
bronze_container = "bronze"
silver_container = "silver"

spark.conf.set(
  f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
  dbutils.secrets.get(scope="adls-creds", key="storage-key")
)
bronze_base = f"abfss://{bronze_container}@{storage_account_name}.dfs.core.windows.net/"
silver_base = f"abfss://{silver_container}@{storage_account_name}.dfs.core.windows.net/"

current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')